In [2]:
 """
@file CS2311_project.ipynb
@brief Code for scene geometry estimator
"""
import sys
import math
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow
from scipy.linalg import polar
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import csv
from PIL import ImageFont, ImageDraw, Image
from google.colab import files

slope_tol = 5
line_pixel_tol = 20
discont_tol = 500
line_thresh =80
font = cv.FONT_HERSHEY_SIMPLEX
line_quads =[]
qd =[]
fname = ""
interactive_mode = True
l11 = 0
l21 = 0
l22 = 0

def edge_line_detect(source):
  global line_thresh
  scale = np.minimum(800/source.shape[1], 600/source.shape[0]) # percent of original size
  width = int(source.shape[1] * scale )
  height = int(source.shape[0] * scale)
  dim = (width, height)
    
  # resize image
  src = cv.resize(source, dim, interpolation = cv.INTER_AREA)
  dst = cv.Canny(src, 50, 200, None, 3)

  # Copy edges to the images that will display the results in BGR
  cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
  cdst1 = np.copy(cdst)
  cdstP = np.copy(cdst)
  cdstQ = np.copy(cdst)
  lines = cv.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 10)

  if lines is not None:
      for i in range(0, len(lines)):
          rho = lines[i][0][0]
          theta = lines[i][0][1]
          a = math.cos(theta)
          b = math.sin(theta)
          x0 = a * rho
          y0 = b * rho
          pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
          pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
          #cv.line(cdst1, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
  if interactive_mode == True:
    cv2_imshow( cdst1)

  linesP = cv.HoughLinesP(dst, 1, 0.5*np.pi / 180, line_thresh, None, 50, 10)
  if linesP is not None:
      for i in range(0, len(linesP)):
          l = linesP[i][0]
          cv.line(cdst, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)
          cv.putText(cdst, str(i), (int((l[0]+l[2])/2), int((l[1]+l[3])/2)), font,0.5, (255,255,255),2,cv.LINE_AA)
  if interactive_mode == True:
    cv2_imshow( cdst)
  return linesP, cdstP, cdstQ

def manipulate_lines(linesP, cdstP, cdstQ):
  # "Source", "Detected Lines (in red) - Standard Hough Line Transform", "Detected Lines (in red) - Probabilistic Line Transform",
  #cv2_imshow( src)
  #cv2_imshow(cdst)
  # Find all lines that make up a quadrilateral
  global line_quads, interactive_mode
  linesP1 = linesP
  slope = np.zeros(linesP.shape[0])
  line_pairs =[]
  line_triplets =[]
  #line_quads = []
  lcnt = linesP.shape[0]
  global fname

  if linesP is not None:
      for i in range(0, len(linesP)):
        l1 = linesP1[i][0]
        #slope[i] = (l1[3] - l1[1])/(l1[2] - l1[0])
        #print(l1, slope1, i)
        for j in range(i+1, len(linesP)):
          #see if two pairs have common endpoint
          l2 = linesP[j][0]
          ep_ij, ord_ij  = line_intersects(l1, l2)
          if np.abs(line_slope(l1)-line_slope(l2)) > slope_tol and ep_ij :
            line_pairs.append([i, j])
            #print(l1, l2, ep_ij, i, j)
            for k in range(0, len(linesP)):
              if(k == i or k == j ):
                  continue
              l3 = linesP[k][0]
              ep_ijk, ord_ijk = line_intersects(ep_ij, l3)
              if np.abs(line_slope(l3)-line_slope(ep_ij)) > slope_tol and ep_ijk:
                if ord_ijk[0] == 1: 
                  line_triplet = [k,i,j]
                else:
                  line_triplet = [i,j,k]
                line_triplets.append(line_triplet)
                #print(l1, l2, l3, ep_ij, ep_ijk, i, j, k)
                for l in range(0, len(linesP)):
                  if(l == i or l == j or l == k):
                     continue
                  l4 = linesP[l][0]
                  ep_ijkl, ord_ijkl = line_intersects(ep_ijk, l4)
                  if  slope_tol and ep_ijkl: #np.abs(line_slope(l4)-line_slope(ep_ijk)) >
                    dist = point_distance(ep_ijkl[0:2], ep_ijkl[2:4])
                    slope_diff = np.abs(line_slope(l4) - line_slope(ep_ijk))
                    #print(dist, slope_diff, i,j, k,l)
                    if ord_ijkl[0] == 1 and dist < discont_tol and slope_diff <slope_tol:
                      line_quads.append([line_triplet[0],line_triplet[1],line_triplet[2], l])
                      #print(ep_ijkl)
                    elif  dist < discont_tol and slope_diff <slope_tol:
                      line_quads.append([l, line_triplet[0],line_triplet[1],line_triplet[2]])
                      #print(ep_ijkl)
  print(line_pairs)
  print(line_triplets)
  print(line_triplets)
  print(line_quads)
  #check(fname)
  print(line_quads)
  #line_quads.clear()
  if interactive_mode == False:
    line_quads.clear()
    line_quads.append(qd)
  #line_quads.append([6,4,3,11])
  lq = np.array(line_quads)
  #print(line_quads)
  #filter duplicate quads
  invalid_list = np.zeros(len(line_quads))
  if line_quads is not None:
      for i in range(0, len(line_quads)):
        for j in range(i+1, len(line_quads)):
          q1 = sorted(line_quads[i])
          q2 = sorted(line_quads[j])
          if q1 == q2:
            invalid_list[j] = True
            #print("Dupe", i, j)
      for i in range(0, len(line_quads)): 
        if invalid_list[i]:
          continue
        m = [0, 0, 0, 0]
        for j in range(4):
          m[j] = line_slope(linesP[line_quads[i][j]][0])
        for j in range(4):
          if np.abs(np.tan((np.pi/180)*(m[(j+1)%4]-m[j]))) < 0.2:
            invalid_list[i] = True
            print("Too oblique", line_quads[i])
        
  print(invalid_list)
  if line_quads is not None:
      for i in range(0, len(line_quads)):
        if invalid_list[i] == True:
          continue
        citer = np.copy(cdstP)
        citer1 = np.copy(cdstQ)
        quad = line_quads[i]
        print(quad)
        q_vert = []
        q_vp = []
        for j in range(4):
          line_col = (np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255))
          l = linesP[quad[j]][0]
          cv.line(citer, (l[0], l[1]), (l[2], l[3]), line_col, 3, cv.LINE_AA)
          cv.putText(citer, str(quad[j]), (int((l[0]+l[2])/2), int((l[1]+l[3])/2)), font,1, (255,255,255),2,cv.LINE_AA)
          #Find precise vertices from line interesections as some corners may not be fully formed
          q_vert.append(line_intersect_point(linesP[quad[j]][0], linesP[quad[(j+1)%4]][0]))
          if j < 2:
            q_vp.append(line_intersect_point(linesP[quad[j]][0], linesP[quad[(j+2)]][0]))
        #print(q_vert, q_vp)
        # Find lowest left vertex
        qa = quad_area(q_vert) / (cdstP.shape[0] *cdstP.shape[1])
        print("Quad area:", qa)
        if(qa <0.075):
          print("Skip this quadrilateral, too small", qa)
          #continue
        min, temp1 = 1e8,1E8
        lv, length, wid,temp = 0, 0, 0,0
        qo_v = []
        asp_ratio = [1.778 ]#1.333, 2.333, , 1.6
        for j in range(4):
          if point_distance([0,0], q_vert[j]) < min:
            lv = j
            min = point_distance([0,0], q_vert[j])
        #print(lv)
        #Now make a rectangle at lowest left corner
        length = np.maximum(point_distance(q_vert[lv], q_vert[(lv+1)%4]) , point_distance(q_vert[(lv+2)%4], q_vert[(lv+3)%4])) 
        wid = np.maximum(point_distance(q_vert[lv], q_vert[(lv+3)%4]), point_distance(q_vert[(lv+1)%4], q_vert[(lv+2)%4])) 
        print("Aspect ratio", "Persp",  length, wid)
        
        if wid>length:
          length, wid = wid, length
        temp  = length/wid
        for i in asp_ratio:
          if temp1 > abs(temp-i):
            asp = i
            temp1 = abs(temp-i)
        print("Aspect ratio", "Persp", asp,temp, length, wid)
        #wid = length/asp
        length = wid*asp
        qo_v.append(q_vert[lv])
        qo_v.append([q_vert[lv][0] + length, q_vert[lv][1]])
        qo_v.append([q_vert[lv][0] + length, q_vert[lv][1]+wid])
        qo_v.append([q_vert[lv][0], q_vert[lv][1]+wid])
        print("qvert", q_vert, qo_v)
        
        #get_homography(q_vert, qo_v)
        
        #print("lv",lv)
        if(point_distance(q_vert[lv],q_vert[(lv+1)%4])<point_distance(q_vert[lv],q_vert[(lv+3)%4])):
          q_vert[(lv+3)%4], q_vert[(lv+1)%4] = q_vert[(lv+1)%4], q_vert[(lv+3)%4]
        #print("q_vert", q_vert)
        q_src = np.array(q_vert)
        #print("bef q_src",q_src)
        q_t = np.zeros_like(q_src)
        q_t[0,:] = q_src[lv,:]
        q_t[1,:] = q_src[(lv+1)%4,:]
        q_t[2,:] = q_src[(lv+2)%4,:]
        q_t[3,:] = q_src[(lv+3)%4,:]
        q_src  = q_t
        q_dst = np.array(qo_v)
        q_dst1 = np.zeros_like(q_dst)
        q_src1 = np.zeros_like(q_src)
        q_dst1[:,0] = q_dst[:,0] - np.mean(q_dst[:,0]) # q_dst[0,0]# 
        q_dst1[:,1] = q_dst[:,1] - np.mean(q_dst[:,1]) #q_dst[0,1]#  
        q_src1[:,0] = q_src[:,0] - np.mean(q_src[:,0]) # q_src[0,0]# 
        q_src1[:,1] = q_src[:,1] - np.mean(q_src[:,1]) #q_src[0,1]# 
        q_src1[:,1] = -q_src1[:,1]
        q_dst1[:,1] = -q_dst1[:,1]
        if(q_src1[1,1] < 0 and q_src1[1,0] < 0):
          q_src1[[1, 3]] = q_src1[[3, 1]]
        
        print("q_src1", q_src1,"\nq_dst1", q_dst1)
        M_h = cv.findHomography(q_dst1, q_src1)[0]
        print("Homography matrix is:", M_h)

        n_vp = cross_vp(q_vp[0], q_vp[1])
        slope = n_vp[1]/n_vp[0]
        q_ctr = np.mean(np.array(q_vert), axis=0)
        theta_sign, phi_sign,domi = locate_vps(q_vp, q_ctr)
        n2 = np.array([slope, -1, -slope*q_ctr[0]+q_ctr[1]])
        ip = cross_prod(n2, n_vp)
        ip = ip/ip[2]
        print("Vanishing points:",q_vp[0], q_vp[1], ip)
        cv.line(citer1, (round(ip[0]), round(ip[1])), (round(q_ctr[0]), round(q_ctr[1])), (0,0,255), 3, cv.LINE_AA)
        vp_n_angle = line_slope([q_ctr[0], q_ctr[1], ip[0],ip[1]])
        if(q_ctr[0] >= ip[0]):
          vp_n_angle += 180
        theta_est = (vp_n_angle - 180) %360
        print("theta_est", theta_est, "vp_n_angle", vp_n_angle)

        q_est = np.concatenate((q_dst1,np.ones((4,1))),axis=1)
        print(q_est)
        q_est = M_h @ q_est.T
        q_est = q_est/q_est[2,:]
        
        homo_plot(q_dst1, q_src1,q_est)
        q_est[1,:] = -q_est[1,:]
        q_est[0,:] += q_ctr[0]
        q_est[1,:] += q_ctr[1]
        print("q_est:\n",q_est)
        #decomposeH(M_h)
        theta, phi = UL_decomp(M_h, theta_sign, phi_sign,domi)
        #if np.isfinite(M_h.any()) and not np.isNan(M_h.any()):
        #decomposeH_LP(M_h)
        #print("----")
        #print(q_src.shape)
        q_dh = M_h @ np.concatenate((q_src, np.ones((4,1))), axis=1).T
        print(q_dh/q_dh[2,:], q_dst)
        norm_dir = np.cross(M_h[0,:], M_h[1,:])
        norm_dir = norm_dir/norm_dir[2]
        #print(q_vert, qo_v)

        print(M_h)

        line_col = (255,255,255)#(np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255))
        line_col1 = (0,255,0)
        line_col2 = (0,0,255)
        for j in range(4):#and np.isfinite(q_vert):
          cv.line(citer1, (round(q_vert[j][0]), round(q_vert[j][1])), (round(q_vert[(j+1)%4][0]), round(q_vert[(j+1)%4][1])), line_col, 3, cv.LINE_AA)
          cv.line(citer, (round(q_vert[j][0]), round(q_vert[j][1])), (round(q_vert[(j+1)%4][0]), round(q_vert[(j+1)%4][1])), line_col, 3, cv.LINE_AA)
        line_col = (np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255))
        for j in range(4):
          cv.line(citer1, (round(qo_v[j][0]), round(qo_v[j][1])), (round(qo_v[(j+1)%4][0]), round(qo_v[(j+1)%4][1])), line_col1, 3, cv.LINE_AA)
          cv.line(citer1, (round(q_est[0][j]), round(q_est[1][j])), (round(q_est[0][(j+1)%4]), round(q_est[1][(j+1)%4])), line_col2, 3, cv.LINE_AA)
          #cv.putText(cdstQ, str(quad[j]), (int((l[0]+l[2])/2), int((l[1]+l[3])/2)), font,1, (255,255,255),2,cv.LINE_AA)
        #cv.putText(citer1, str("theta: ") + str(round(theta, 1)) , (int(q_ctr[0]), int(q_ctr[1]-100)), font,1.5, (255,255,255),5,cv.LINE_AA)
        #cv.putText(citer1, str("phi: ")+ str(round(phi, 1)), (int(q_ctr[0]), int(q_ctr[1]-50)), font,1.5, (255,255,255),5,cv.LINE_AA)
        #cv.putText(citer1, str("QA: ")+ str(round(qa, 2)), (int(q_ctr[0]), int(q_ctr[1])), font,1.5, (255,255,255),5,cv.LINE_AA)
        cv.putText(citer1, str("theta: ") + str(round(theta, 1)) , (50, 100), font,1.5, (255,255,255),5,cv.LINE_AA)
        cv.putText(citer1, str("phi: ")+ str(round(phi, 1)), (50, 150), font,1.5, (255,255,255),5,cv.LINE_AA)
        cv.putText(citer1, str("QA: ")+ str(round(qa, 2)), (50, 200), font,1.5, (255,255,255),5,cv.LINE_AA)
        xloc = citer1.shape[0] -200
        yloc = 100
        if False:#(qa<0.075):
          cv.putText(citer1, str("Too far") , (xloc, yloc), font,3, (255,255,255),5,cv.LINE_AA)
        else:
          if np.abs(theta) >20 and np.abs(theta)<70 and  phi> 55 and phi<135:
           cv.putText(citer1, str("Good") , (xloc, yloc), font,3, (255,255,255),5,cv.LINE_AA)
          else:
            cv.putText(citer1, str("Bad") , (xloc, yloc), font,3, (255,255,255),5,cv.LINE_AA)
        get_xform(q_vert)
        #print(q_ctr)
        if interactive_mode == True:
          cv2_imshow( citer)
          cv2_imshow( citer1)
           
  
  #print(line_pairs)
  #print(line_triplets)
  #print(line_quads)
  #cv2_imshow( cdstP)
  #cv2_imshow( cdstQ)

  cv.waitKey()
  return citer1

def locate_vps(vp, qc):
  idx = -1
  pt =-1
  ph = -1
  if (point_distance(vp[0],qc) > point_distance(vp[1], qc)):
    id = 0
  else:
    id = 1
  print("Choose VP#", id, " for VP analysis")
  if (np.abs(vp[id][0] -qc[0]) > np.abs(vp[id][1] -qc[1])):
    print("VP_x is", id,vp[id], qc)
    print("VP_y is", (id+1)%2,vp[(id+1)%2], qc)
    dom = 1
  else:
    print("VP_x is", (id+1)%2,vp[(id+1)%2], qc)
    print("VP_y is", id,vp[id], qc)
    dom = 0

  if vp[(id+1)%2][0] - qc[0] > 0 and dom == 0:
      pt = 1
  elif vp[(id+1)%2][0] - qc[0] < 0 and dom ==0:
      pt = 0
  elif vp[(id+1)%2][1] - qc[1] > 0 and dom == 1:
    ph = 0
  elif vp[(id+1)%2][1] - qc[1] < 0 and dom == 1:
    ph = 1
  print("pt", pt, "ph", ph)
  return pt, ph,dom

def homo_plot(i,o,e):
  global interactive_mode
  if interactive_mode == False:
    return
  plt.plot([i[0,0],i[1,0]],[i[0,1],i[1,1]],'k-')
  plt.plot([i[1,0],i[2,0]],[i[1,1],i[2,1]],'k-')
  plt.plot([i[2,0],i[3,0]],[i[2,1],i[3,1]],'k-')
  plt.plot([i[3,0],i[0,0]],[i[3,1],i[0,1]],'k-')

  plt.plot([o[0,0],o[1,0]],[o[0,1],o[1,1]],'r-')
  plt.plot([o[1,0],o[2,0]],[o[1,1],o[2,1]],'r-')
  plt.plot([o[2,0],o[3,0]],[o[2,1],o[3,1]],'r-')
  plt.plot([o[3,0],o[0,0]],[o[3,1],o[0,1]],'r-')

  plt.plot([e[0,0],e[0,1]],[e[1,0],e[1,1]],'g-')
  plt.plot([e[0,1],e[0,2]],[e[1,1],e[1,2]],'g-')
  plt.plot([e[0,2],e[0,3]],[e[1,2],e[1,3]],'g-')
  plt.plot([e[0,3],e[0,0]],[e[1,3],e[1,0]],'g-')

  plt.axes().set_aspect('equal', 'datalim')
  plt.show()


def func(x):
    #print("l11", l11, l22, l21)
    return [l22*np.cos(x[0]) - l11*np.sin(x[1]),
            l22*np.sin(x[0])*np.cos(x[1]) + l21*np.sin(x[1])]

def UL_decomp(M,pm,ph,d):
  global l11, l22, l21
  b1 = np.mat([[1, 0, -M[0,2]/M[2,2]],
           [0,1, -M[1,2]/M[2,2]],
           [0,0,1]])
  M1 = b1 @ M
  b2 = np.mat([[1, -M1[0,1]/M1[1,1],0],
          [0,1, 0],
          [0,0,1]])
  l = b2 @ M1
  u = np.linalg.inv(b1) @ np.linalg.inv(b2)
  print("u\n", u)
  print("l\n", l)
  print("u@l", u@l)
  theta_hat = np.arcsin(np.sqrt(-(l[1,0]*l[2,0])/(l[1,1]*l[2,1])))*180/np.pi
  phi_hat = np.arctan((-l[1,1]/l[1,0])*np.sin(theta_hat*np.pi/180))*180/np.pi
  l11 = l[0,0]
  l22 = l[1,1]
  l21 = l[1,0]
  print("l11, l22, l21", l11, l22, l21)
  root = fsolve(func, [0, 0])
  theta_hat = root[0]*180/np.pi
  phi_hat = root[1]*180/np.pi
  if(theta_hat< -90 ):
    theta_hat = theta_hat % 180
    phi_hat =   phi_hat%180  
  print("Theta:", theta_hat ,"Phi:",  phi_hat)
  print("pm",pm, "ph",ph, "dominant", d)
  if pm == 0 and theta_hat < 0:
    theta_hat = -theta_hat
    phi_hat = 180-phi_hat
  if pm == 1 and theta_hat > 0:
    theta_hat = -theta_hat
    phi_hat = 180-phi_hat
  if ph == 0 and phi_hat >90:
    theta_hat = -theta_hat
    phi_hat = 180-phi_hat
  if  ph == 1 and phi_hat < 90:
    theta_hat = -theta_hat
    phi_hat = 180-phi_hat

  print("theta_hat:", theta_hat)
  print("phi_hat:", phi_hat)
  print(root)
  return theta_hat, phi_hat

def decomposeH(H):
  norm = np.sqrt(H[0,0]**2 + H[1,0]**2 + H[2,0]**2)
  H = H/norm
  r3 = np.cross(H[:,0], H[:,1])
  t = H[:,2]
  R = np.zeros((3,3)) 
  R[:,0] = H[:,0]
  R[:,1] = H[:,1]
  R[:,2] = r3
  x_rot = np.arctan2(R[2,1], R[2,2]) * 180 / np.pi
  y_rot = np.arctan2(-R[2,0], np.sqrt(R[2,1]**2 + R[2,2]**2)) * 180 / np.pi
  z_rot = np.arctan2(R[1,0], R[0,0]) * 180 / np.pi
  print("X-rotation", x_rot, "Y-rotation", y_rot,"Z-rotation", z_rot,)
  print("Rotation matrix is: \n", R)
  print("Translation matrix is:", t)

def check(fn):
  global line_quads
  with open('drive/MyDrive/quad.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    filename = fn.split('.')
    for row in csv_reader:
      print(row, filename)
      if row[0] == filename[0]:
        print(row[0], row[1], row[2], row[3], row[4])
        line_quads=[]
        line_quads.append([int(row[1]), int(row[2]), int(row[3]), int(row[4])])
        print(line_quads)

def decomposeH_LP(H):
  R,S = polar(H,"left")
  print("Rotation matrix is: \n", R)
  print("Squeeze matrix is:", S)
  x_rot = np.arctan2(R[2,1], R[2,2]) * 180 / np.pi
  y_rot = np.arctan2(-R[2,0], np.sqrt(R[2,1]**2 + R[2,2]**2)) * 180 / np.pi
  z_rot = np.arctan2(R[1,0], R[0,0]) * 180 / np.pi
  print("X-rotation", x_rot, "Y-rotation", y_rot,"Z-rotation", z_rot,)

def get_homography(q1, q2):
  M1 = get_xform(q1)
  M2 = get_xform(q2)
  M  = np.linalg.inv(M1) @ M2
  #print("q1", q1)
  #print("q1", q2)
  p = np.zeros((4,3))
  p[:,:2] = np.mat(q1)
  p[:,2] = 1
  #print(p)
  #print("M at q2", p @ M)
  return M

def get_xform(q):
  M = np.linalg.inv(np.mat([[q[0][0], q[0][1], 1], \
              [q[1][0], q[1][1], 1], \
              [q[2][0], q[2][1], 1]]))
  d = np.mat([q[3][0], q[3][1], 1])
  d = d @ M
  D = np.diagflat(d)
  M = D @ M
  return M

def quad_area(q):
  q = np.array(q)
  print(q.shape)
  t1 = np.concatenate((q[:3,:],np.ones((3,1))),axis=1)
  a1 = np.abs(np.linalg.det(t1))
  #print("t1", t1)
  t1 = np.concatenate((q[2:,:],np.ones((2,1))),axis=1)
  t1  = np.concatenate((t1,np.mat([q[0,0], q[0,1],1])), axis =0)
  #print("t1", t1)
  a2 = np.abs(np.linalg.det(t1))
  #print("A1:", a1, " A2",a2)
  return (a1+a2)

def cross_prod(p1, p2):
  x_vp = np.cross(p1, p2)
  return x_vp

def cross_vp(p1, p2):
  vp1 = np.array([p1[0],p1[1], 1])
  vp2 = np.array([p2[0],p2[1], 1])
  x_vp = np.cross(vp1, vp2)
  #x_vp = (x_vp/x_vp[2])[:2]
  return x_vp

def point_distance(p1, p2):
  return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 )

def line_slope(l):
    if(l[2] - l[0] == 0): 
      return 90
    else:
      return (180/np.pi)*np.arctan((-l[3] + l[1])/(l[2] - l[0]))

def line_intersect_point(l1,l2):
  #form lines thru two points, then find intersection points
  p1 = np.array([l1[0], l1[1], 1])
  p2 = np.array([l1[2], l1[3], 1])
  line1 = np.cross(p1,p2)
  p1 = np.array([l2[0], l2[1], 1])
  p2 = np.array([l2[2], l2[3], 1])
  line2 = np.cross(p1,p2)
  ip = np.cross(line1,line2)
  if ip[2] != 0:
    ip = (ip/ip[2])[:2]
  else:
    print("parallel Lines", l1, l2, ip)
    ip = ip[:2]
  #print(l1, l2, ip)
  return ip.tolist()

def line_intersects(l1, l2):
  if np.allclose(l1[:2],l2[:2],1e-15, line_pixel_tol) :
    return ([l1[2], l1[3], l2[2], l2[3]], [1, 1])
  elif np.allclose(l1[:2],l2[2:4],1e-15, line_pixel_tol) :
    return ([l1[2], l1[3], l2[0], l2[1]],[1, 0])
  elif np.allclose(l1[2:4],l2[:2],1e-15, line_pixel_tol) :
    return ([l1[0], l1[1], l2[2], l2[3]], [0, 1])
  elif np.allclose(l1[2:4],l2[2:4],1e-15, line_pixel_tol) :
    return ([l1[0], l1[1], l2[0], l2[1]], [0, 0])
  else:
    return (False, [0,0])
def driver():
    global line_quads, qd, interactive_mode,line_thresh
    interactive_mode = False
    with open('drive/MyDrive/quad.csv') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      line_count = 0
      for row in csv_reader:
          src = cv.imread('drive/MyDrive/pics/meas/'+row[0]+'.jpg', cv.IMREAD_GRAYSCALE)
          if src is None:
            print ('Error opening image!',row[0])
            print ('Usage: hough_lines.py [image_name -- default ' + row[0] + '] \n')
          else:
            print("Working on file:" + row[0]+".jpg")
            if len(row) == 5:
              line_thresh =80
            else:
              line_thresh = int(row[5])
            qd = [int(row[1]), int(row[2]), int(row[3]), int(row[4])]
            print(row[0], row[1], row[2], row[3], row[4])
            lines,cp, cq = edge_line_detect(src)
            im = manipulate_lines(lines,cp, cq)
            cv.imwrite("drive/MyDrive/pics/meas/res1/"+row[0]+".jpg",im)

def interactive():           
    #### Active code
    global line_thresh 
    #line_thresh =40
    #line_quads.append([52, 0, 61, 53])
    src = cv.imread('drive/MyDrive/pics/meas/4p5_45_90.jpg', cv.IMREAD_GRAYSCALE)
    # Check if image is loaded fine
    if src is None:
        print ('Error opening image!')
        print ('Usage: hough_lines.py [image_name -- default ' + default_file + '] \n')
    else:
      lines,cp, cq = edge_line_detect(src)
      manipulate_lines(lines,cp, cq)

#def main(argv):
#    filename = argv[0] if len(argv) > 0 else default_file
#    # Loads an image   cam_view.png  monitor.jpg  nsb.png vik_mon.png  cs231a_p1.jpg
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
driver()
#interactive()
#uploaded = files.upload()
#fname  = next(iter(uploaded))
#src = cv.imread(fname, cv.IMREAD_GRAYSCALE)


#if __name__ == "__main__":
#  main(sys.argv[1:])

Mounted at /content/drive
Working on file:4p5_45_90.jpg
4p5_45_90  3  6  13  16
[[2, 9], [3, 5], [3, 6], [3, 14], [3, 16], [5, 12], [5, 13], [6, 12], [6, 13], [9, 29], [12, 14], [12, 16], [13, 14], [13, 16], [14, 15], [14, 16], [15, 16], [17, 20], [20, 22], [21, 26], [22, 31], [24, 26]]
[[3, 5, 6], [12, 3, 5], [3, 5, 13], [14, 3, 5], [16, 3, 5], [3, 6, 5], [12, 3, 6], [3, 6, 13], [14, 3, 6], [16, 3, 6], [5, 3, 14], [6, 3, 14], [12, 3, 14], [3, 14, 13], [3, 14, 15], [3, 14, 16], [5, 3, 16], [6, 3, 16], [12, 3, 16], [3, 16, 13], [3, 16, 14], [3, 16, 15], [5, 12, 3], [6, 5, 12], [13, 5, 12], [5, 12, 14], [5, 12, 16], [3, 5, 13], [6, 5, 13], [12, 5, 13], [5, 13, 14], [5, 13, 15], [5, 13, 16], [6, 12, 3], [5, 6, 12], [13, 6, 12], [6, 12, 14], [6, 12, 16], [3, 6, 13], [5, 6, 13], [12, 6, 13], [6, 13, 14], [6, 13, 15], [6, 13, 16], [3, 12, 14], [5, 12, 14], [6, 12, 14], [12, 14, 13], [12, 14, 15], [12, 14, 16], [3, 12, 16], [5, 12, 16], [6, 12, 16], [12, 16, 13], [12, 16, 14], [12, 16, 15], [

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:398: RuntimeWarning: invalid value encountered in sqrt


Working on file:4p5_45_120.jpg
4p5_45_120  4  0  3  2
[[0, 3], [0, 4], [0, 6], [1, 4], [1, 6], [2, 3], [2, 6], [3, 5], [5, 6]]
[[0, 3, 2], [4, 0, 3], [0, 3, 5], [0, 3, 6], [0, 4, 1], [3, 0, 4], [6, 0, 4], [0, 6, 1], [0, 6, 2], [0, 6, 3], [4, 0, 6], [0, 6, 5], [1, 4, 0], [2, 1, 4], [5, 1, 4], [6, 1, 4], [1, 6, 0], [1, 6, 3], [4, 1, 6], [2, 3, 0], [2, 3, 6], [2, 6, 0], [2, 6, 3], [0, 3, 5], [6, 3, 5], [5, 6, 0], [5, 6, 3]]
[[0, 3, 2], [4, 0, 3], [0, 3, 5], [0, 3, 6], [0, 4, 1], [3, 0, 4], [6, 0, 4], [0, 6, 1], [0, 6, 2], [0, 6, 3], [4, 0, 6], [0, 6, 5], [1, 4, 0], [2, 1, 4], [5, 1, 4], [6, 1, 4], [1, 6, 0], [1, 6, 3], [4, 1, 6], [2, 3, 0], [2, 3, 6], [2, 6, 0], [2, 6, 3], [0, 3, 5], [6, 3, 5], [5, 6, 0], [5, 6, 3]]
[[3, 1, 2, 4], [1, 4, 0, 3], [4, 0, 3, 2], [4, 0, 3, 5], [3, 0, 4, 1], [2, 3, 0, 4], [5, 3, 0, 4], [6, 0, 4, 1], [2, 6, 0, 4], [5, 6, 0, 4], [0, 6, 1, 4], [4, 0, 6, 1], [4, 0, 6, 2], [4, 0, 6, 5], [6, 1, 4, 0], [1, 6, 0, 4], [2, 1, 6, 3], [5, 1, 6, 3], [4, 1, 6, 0], [1, 2, 3, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:398: RuntimeWarning: invalid value encountered in arcsin


Working on file:4p5_m60_90.jpg
4p5_m60_90 37 17 60 10
[[1, 36], [1, 56], [1, 59], [1, 67], [1, 74], [3, 25], [3, 42], [3, 51], [3, 58], [4, 63], [5, 13], [6, 33], [6, 46], [6, 63], [6, 72], [7, 48], [7, 50], [7, 62], [7, 66], [10, 60], [11, 33], [11, 42], [11, 45], [11, 60], [11, 61], [13, 42], [15, 33], [15, 42], [15, 45], [15, 46], [15, 52], [16, 49], [17, 37], [17, 53], [18, 63], [19, 32], [19, 40], [23, 39], [23, 60], [23, 73], [24, 48], [24, 73], [25, 49], [26, 66], [27, 33], [27, 42], [27, 45], [27, 60], [27, 61], [28, 39], [28, 60], [33, 45], [33, 52], [33, 72], [38, 51], [38, 58], [39, 60], [42, 51], [42, 52], [42, 58], [43, 47], [44, 71], [45, 52], [45, 64], [46, 51], [46, 52], [46, 58], [48, 69], [52, 60], [52, 61], [61, 64], [63, 65]]
[[34, 1, 36], [1, 36, 59], [1, 36, 67], [34, 1, 56], [1, 56, 57], [1, 56, 67], [1, 56, 74], [34, 1, 59], [1, 59, 36], [1, 59, 67], [1, 59, 74], [34, 1, 67], [1, 67, 36], [1, 67, 56], [1, 67, 57], [1, 67, 59], [1, 67, 74], [1, 67, 75], [34, 1, 7

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:398: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:399: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Working on file:3_45_90.jpg
3_45_90 3  13  7  35
[[3, 13], [3, 20], [3, 35], [3, 40], [5, 29], [5, 38], [7, 9], [7, 13], [9, 15], [13, 15], [17, 29], [23, 24], [29, 31], [31, 38], [39, 43]]
[[3, 13, 7], [3, 13, 9], [3, 13, 15], [20, 3, 13], [35, 3, 13], [13, 3, 20], [40, 3, 20], [3, 35, 8], [13, 3, 35], [40, 3, 35], [3, 40, 9], [20, 3, 40], [35, 3, 40], [5, 29, 38], [5, 38, 29], [7, 9, 40], [7, 13, 3], [7, 13, 40], [40, 9, 15], [3, 13, 15], [40, 13, 15], [17, 29, 38], [23, 24, 1], [38, 29, 31], [31, 38, 29]]
[[3, 13, 7], [3, 13, 9], [3, 13, 15], [20, 3, 13], [35, 3, 13], [13, 3, 20], [40, 3, 20], [3, 35, 8], [13, 3, 35], [40, 3, 35], [3, 40, 9], [20, 3, 40], [35, 3, 40], [5, 29, 38], [5, 38, 29], [7, 9, 40], [7, 13, 3], [7, 13, 40], [40, 9, 15], [3, 13, 15], [40, 13, 15], [17, 29, 38], [23, 24, 1], [38, 29, 31], [31, 38, 29]]
[[4, 3, 0, 6], [3, 13, 7, 20], [3, 13, 7, 35], [31, 5, 29, 38], [17, 5, 38, 29], [31, 5, 38, 29], [20, 7, 13, 3], [35, 7, 13, 3], [5, 38, 29, 31], [5, 31, 38, 29]